# chart module

This module relies on `pygal` library, so the returned charts are instances of `pygal.chart`. See options at 
[pygal site][1]

I made a JavaScript 'equivalent': https://code.earthengine.google.com/b2922b860b85c1120250794fb82dfda8

  [1]: http://www.pygal.org/en/latest/documentation/index.html

In [1]:
import ee
ee.Initialize()

In [2]:
from geetools import tools, ui

In [3]:
test_site = ee.Geometry.Point([-71, -42])

In [4]:
test_feat = ee.Feature(test_site, {'name': 'test feature', 'buffer':0})

In [5]:
test_featcol = ee.FeatureCollection([
    test_feat, 
    test_feat.buffer(100).set('name', 'buffer 100', 'buffer', 100),
    test_feat.buffer(1000).set('name', 'buffer 1000', 'buffer', 1000)
])

## Time Series

In [6]:
col = ee.ImageCollection('COPERNICUS/S2').filterBounds(test_site)

In [7]:
time_series = col.filterDate('2018-01-01', '2018-04-01')

In [8]:
bands = ['B1', 'B2', 'B3']

## **Image.series**
Chart band values across images in relation with a property or a band

In [9]:
chart_ts = ui.chart.Image.series(**{
    'imageCollection': time_series, 
    'region': test_site,
    'scale': 10,
    'bands': bands,
    'label_bands':['band B1', 'B2 band', 'this is B3'],
    'properties':['CLOUD_COVERAGE_ASSESSMENT'],
    'label_properties':['CLOUD_COVER']
})

In [10]:
chart_ts.render_widget(width='50%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [11]:
chart_ts.dataframe

,this is B3,band B1,B2 band,CLOUD_COVER
2018-01-01 14:37:52.570,2673.0,3622.0,2749.0,88.2101
2018-01-06 14:43:00.500,6483.0,6989.0,6841.0,98.1513
2018-01-16 14:38:41.740,1012.0,1196.0,1042.0,0.0646
2018-01-21 14:41:57.890,1035.0,1232.0,1070.0,58.2937
2018-01-26 14:40:02.120,1038.0,1210.0,1082.0,0.2448
2018-02-05 14:36:43.690,1037.0,1196.0,1072.0,0.0204
2018-02-10 14:42:58.290,4290.0,4730.0,4153.0,30.9238
2018-02-15 14:40:26.370,1020.0,1212.0,1069.0,0.0000
2018-02-20 14:33:53.510,1036.0,1230.0,1077.0,0.0121
2018-02-25 14:37:50.240,1019.0,1234.0,1091.0,0.4717


## **Image.seriesByRegion**
Chart the value of one band in different regions in relation with a property or a band

In [12]:
chart_ts_region = ui.chart.Image.seriesByRegion(**{
    'imageCollection': time_series,
    'reducer': ee.Reducer.median(),
    'regions': test_featcol,
    'scale': 10,
    'band': 'B11',
    'seriesProperty': 'name'
})

In [13]:
chart_ts_region.render_widget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [14]:
chart_ts.dataframe

,this is B3,band B1,B2 band,CLOUD_COVER
2018-01-01 14:37:52.570,2673.0,3622.0,2749.0,88.2101
2018-01-06 14:43:00.500,6483.0,6989.0,6841.0,98.1513
2018-01-16 14:38:41.740,1012.0,1196.0,1042.0,0.0646
2018-01-21 14:41:57.890,1035.0,1232.0,1070.0,58.2937
2018-01-26 14:40:02.120,1038.0,1210.0,1082.0,0.2448
2018-02-05 14:36:43.690,1037.0,1196.0,1072.0,0.0204
2018-02-10 14:42:58.290,4290.0,4730.0,4153.0,30.9238
2018-02-15 14:40:26.370,1020.0,1212.0,1069.0,0.0000
2018-02-20 14:33:53.510,1036.0,1230.0,1077.0,0.0121
2018-02-25 14:37:50.240,1019.0,1234.0,1091.0,0.4717


## Chart band values in one image across features in relation with a property value or a band

In [23]:
chart_band = ui.chart.Image.bandsByRegion(**{
    'image': ee.Image(time_series.first()), 
    'collection': test_featcol,
    'reducer': 'mean',
    'scale': 10,
    'bands': ['B1', 'B2', 'B3'],
    'xProperty': 'buffer', # You can use a band too!
    'labels': ['B1 mean', 'B2 mean', 'B3 mean']
})

In [24]:
chart_band.render_widget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [25]:
chart_band.dataframe

,B3 mean,B2 mean,B1 mean
0,2673.000000,2749.000000,3622.000000
100,2665.863711,2781.726426,3547.196452
1000,2811.374592,2983.575209,3373.623957


## Concatenate Charts

### Concatenate mean with median

In [26]:
chart_band_median = ui.chart.Image.bandsByRegion(**{
    'image': ee.Image(time_series.first()), 
    'collection': test_featcol,
    'reducer': 'median',
    'scale': 10,
    'bands': ['B1', 'B2', 'B3'],
    'xProperty': 'buffer', # You can use a band too!
    'labels': ['B1 median', 'B2 median', 'B3 median']
})

In [27]:
chart_band_median.render_widget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [28]:
median_mean = chart_band_median.cat(chart_band)

In [29]:
median_mean.render_widget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [30]:
median_mean.dataframe

,B3 mean,B1 median,B2 mean,B3 median,B2 median,B1 mean
0,2673.000000,3622.000,2749.000000,2673.000000,2749.000000,3622.000000
100,2665.863711,3563.000,2781.726426,2640.666667,2781.000000,3547.196452
1000,2811.374592,3496.375,2983.575209,2854.880285,2983.859858,3373.623957
